Imports

In [1]:
import os
import glob
from datetime import datetime, date
import numpy as np
import pandas as pd

We use `glob` to iterate through all 12 csv files (one for each month) 
and create a list of dataframes. We concatenate these dataframes 
together to create a single dataframe `df` for the past year.

In [4]:
dir = os.path.abspath("")
csv_folder = "/".join(dir.split("\\")[:-1] + ["02_Prepare", "CSVs", "*"])

csv_list = []
for csv in glob.glob(csv_folder):
    csv_list.append(pd.read_csv(csv, index_col=None, header=0))

df = pd.concat(csv_list, axis=0, ignore_index=True)

del dir, csv_folder, csv_list, csv

We check the `shape` and `dtypes` attributes of the dataframe for meta 
information of the dataframe.

In [ ]:
df.shape, df.dtypes

We change `"started_at"` and `"ended_at"` columns to be datetimes 
instead of strings. We then create a `"ride_length"` column which is the
time length of the ride in minutes.

In [ ]:
f = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S")
df["started_at"] = df["started_at"].apply(f)
df["ended_at"] = df["ended_at"].apply(f)

df["ride_length"] = df["ended_at"] - df["started_at"]
df["ride_length"] = df["ride_length"].apply(lambda x: x.total_seconds() / 60)


del f

Creating `"season"`, `"month"`, `"day_of_week"`, and `"start_hour"` 
columns based on the `"started_at"` column. Althought most of these may 
not be necessary to create, having them will speed up some analysis.

In [ ]:
# Season Column ---------------------------------------------------------------
Y = 2000  # Dummy leap year to allow input X-02-29 (leap day)
seasons = [('winter', (date(Y,  1,  1),  date(Y,  3, 20))),
           ('spring', (date(Y,  3, 21),  date(Y,  6, 20))),
           ('summer', (date(Y,  6, 21),  date(Y,  9, 22))),
           ('autumn', (date(Y,  9, 23),  date(Y, 12, 20))),
           ('winter', (date(Y, 12, 21),  date(Y, 12, 31)))]

def get_season(now):
    """Return current nothern hemisphere season"""
    if isinstance(now, datetime):
        now = now.date()
    now = now.replace(year=Y)
    return next(season for season, (start, end) in seasons
                if start <= now <= end)

df["season"] = df["started_at"].apply(lambda x: get_season(x))


# Other columns ---------------------------------------------------------------
df["month"] = df["started_at"].apply(lambda x: datetime.strftime(x, "%m"))
df['day_of_week'] = df['started_at'].apply(lambda x: x.weekday())
df["start_hour"] = df["started_at"].apply(lambda x: datetime.strftime(x, "%H:00"))


del Y, seasons, get_season

Check to see which columns contain null/NaN/na values and how many.

In [5]:
null = df[df.isnull().any(axis=1)]
for column in null.columns:
    print(f"{column}: {null[column].isna().sum()}")

ride_id: 0
rideable_type: 0
started_at: 0
ended_at: 0
start_station_name: 790207
start_station_id: 790204
end_station_name: 843361
end_station_id: 843361
start_lat: 0
start_lng: 0
end_lat: 4766
end_lng: 4766
member_casual: 0
ride_length: 0
day_of_week: 0
start_hour: 0
season: 0
month: 0


was found that every end_lat that had NaN also had NaN for end_station_name and end_station_id, so no way to recover that data. this only made up ~4800 rows out of 5.75 million so i dropped them.

In [6]:
df.drop(df[(df["end_station_name"].isna()) & (df["end_lat"].isna())].index, inplace=True)

In [7]:
df["start_coords"] = list(zip(df.start_lat, df.start_lng))
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,ride_length,day_of_week,start_hour,season,month,start_coords
0,C809ED75D6160B2A,electric_bike,2021-05-30 11:58:15,2021-05-30 12:10:39,NaN,NaN,NaN,NaN,41.90,-87.63,41.89,-87.61,casual,12.400000,6,11:00,spring,05,"(41.9, -87.63)"
1,DD59FDCE0ACACAF3,electric_bike,2021-05-30 11:29:14,2021-05-30 12:14:09,NaN,NaN,NaN,NaN,41.88,-87.62,41.79,-87.58,casual,44.916667,6,11:00,spring,05,"(41.88, -87.62)"
2,0AB83CB88C43EFC2,electric_bike,2021-05-30 14:24:01,2021-05-30 14:25:13,NaN,NaN,NaN,NaN,41.92,-87.70,41.92,-87.70,casual,1.200000,6,14:00,spring,05,"(41.92, -87.7)"
3,7881AC6D39110C60,electric_bike,2021-05-30 14:25:51,2021-05-30 14:41:04,NaN,NaN,NaN,NaN,41.92,-87.70,41.94,-87.69,casual,15.216667,6,14:00,spring,05,"(41.92, -87.7)"
4,853FA701B4582BAF,electric_bike,2021-05-30 18:15:39,2021-05-30 18:22:32,NaN,NaN,NaN,NaN,41.94,-87.69,41.94,-87.70,casual,6.883333,6,18:00,spring,05,"(41.94, -87.69)"


In [9]:
station_coords_count = df.groupby("start_station_name").start_coords.value_counts()

create dict with station names as keys and (lat, long) coords as values. also create a dict with coords as keys and station names as values

In [ ]:
stations = df.start_station_name.value_counts().index.tolist()
stations_coords = {}

for station in stations:
    t = df.loc[df.start_station_name == station]
    stations_coords[station] = (
        t.iloc[0, 8],
        t.iloc[0, 9]
    )

reverse_coords = {values: keys for keys, values in stations_coords.items()}

creates start_coords column and end_coords column.  
uses defined functions to choose the closest station coords based on dicts for rows with NaN for station name(s).  
drops redundant columns, makes all lats and longs for the same for each station observation

In [ ]:
def d_between_points(x, point):
    """Returns distance between x and point using pythag"""
    return np.sqrt((point[0]-x[0])**2 + (point[1]-x[1])**2)

def minimize(x, point_l):
    """Returns point from point_l (point list) that x is closest to"""
    lst = np.array([d_between_points(x, i) for i in point_l])
    idx = lst.argmin()

    return point_l[idx]


df["start_coords"] = list(zip(df["start_lat"], df["start_lng"]))
df["end_coords"] = list(zip(df["end_lat"], df["end_lng"]))

possible_coords = list(reverse_coords.keys())

df.loc[df["start_station_name"].isna(), "start_station_name"] = df.loc[df["start_station_name"].isna(), "start_coords"].apply(lambda x: reverse_coords[minimize(x, point_l=possible_coords)])
df.loc[df["end_station_name"].isna(), "end_station_name"] = df.loc[df["end_station_name"].isna(), "end_coords"].apply(lambda x: reverse_coords[minimize(x, point_l=possible_coords)])

df.drop(["start_station_id", "end_station_id"], axis=1, inplace=True)


df["start_coords"] = df["start_station_name"].apply(lambda x: stations_coords[x])
df["end_coords"] = df["end_station_name"].apply(lambda x: stations_coords[x])

df["start_lat"] = df["start_coords"].apply(lambda x: x[0])
df["start_lng"] = df["start_coords"].apply(lambda x: x[1])
df["end_lat"] = df["end_coords"].apply(lambda x: x[0])
df["end_lng"] = df["end_coords"].apply(lambda x: x[1])


df.drop(["start_coords", "end_coords"], axis=1, inplace=True)

swaps started_at and ended_at for observations with negative ride length (only 140 so i prob could've just ignored but either way it won't make much of a difference.)  
recomputes ride_length as opposed to just multiplying by -1 bc I felt like it

In [ ]:
idx = df["ride_length"] < 0
df.loc[idx, ["started_at", "ended_at"]] = df.loc[idx, ["ended_at", "started_at"]].values

df['ride_length'] = df['ended_at'] - df['started_at']
df['ride_length'] = df['ride_length'].apply(lambda x: x.total_seconds() / 60)


saves dataframe to csv for analysis

In [ ]:
df.to_csv("Clean_Cyclistic_Data.csv")